In [37]:
import time
import pandas as pd
import numpy as np
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

There are many insignificant features to our work, like codes, PK, FK and out of context. 
Based on prior works and experience of especialists, will be taken off nonsense features. 
A study was carried out on the evolution and correspondence of these resources in the ENEM and Census
data for the period 2009-2018. See (featuresOrganisaton.xls)

In [38]:
teacher = ['FK_COD_DOCENTE', 'PK_COD_ENTIDADE','ID_ESPECIALIZACAO','ID_MESTRADO','ID_DOUTORADO']

school = ['ANO_CENSO','PK_COD_ENTIDADE', 'FK_COD_MUNICIPIO','FK_COD_ESTADO','ID_DEPENDENCIA_ADM','ID_AGUA_INEXISTENTE',
                  'ID_ESGOTO_INEXISTENTE','ID_ENERGIA_INEXISTENTE',
                   'ID_LABORATORIO_INFORMATICA','ID_LABORATORIO_CIENCIAS','ID_SALA_ATENDIMENTO_ESPECIAL',
                   'ID_BIBLIOTECA','ID_SALA_LEITURA','ID_SANITARIO_FORA_PREDIO',
                   'ID_SANITARIO_PNE','ID_DEPENDENCIAS_PNE','NUM_SALAS_UTILIZADAS','ID_EQUIP_TV',
                   'ID_EQUIP_DVD','ID_EQUIP_COPIADORA','ID_EQUIP_IMPRESSORA','NUM_COMP_ADMINISTRATIVOS',
                   'NUM_COMP_ALUNOS','ID_INTERNET','ID_BANDA_LARGA','NUM_FUNCIONARIOS','ID_ALIMENTACAO']

enem1 = ['NU_INSCRICAO','NU_ANO','PK_COD_ENTIDADE','IN_TP_ENSINO','IDADE','TP_SEXO','TP_COR_RACA','ST_CONCLUSAO','NU_NT_CN',
        'NU_NT_CH','NU_NT_LC','NU_NT_MT','NU_NOTA_REDACAO']

enem2 = ['NU_INSCRICAO','Q02','Q03','Q01','Q04']

enem= ['NU_ANO','PK_COD_ENTIDADE','IN_TP_ENSINO','IDADE','TP_SEXO','TP_COR_RACA','ST_CONCLUSAO','NU_NT_CN',
        'NU_NT_CH','NU_NT_LC','NU_NT_MT','NU_NOTA_REDACAO', 'Q02','Q03','Q01','Q04'
]




In [39]:
#Students of ENEM
E1 = pd.read_csv('~/data/enem/2011/DADOS/MICRODADOS_ENEM_2011.csv', usecols = enem1)
E2 = pd.read_csv('~/data/enem/2011/DADOS/QUESTIONARIO_ENEM_2011.csv', usecols=enem2)
E = pd.merge(E1, E2, on = 'NU_INSCRICAO', how= "inner")

In [40]:
#School census data
CE =    pd.read_csv("~/data/censo/2011/DADOS/ESCOLAS.CSV", sep='|', encoding="iso-8859-2",usecols= school)

In [41]:
#Teacher census data by Braazilian mesoregion
CD_CO = pd.read_csv("~/data/censo/2011/DADOS/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols= teacher)
CD_NE = pd.read_csv("~/data/censo/2011/DADOS/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols= teacher)
CD_N =  pd.read_csv("~/data/censo/2011/DADOS/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols= teacher)
CD_SE = pd.read_csv("~/data/censo/2011/DADOS/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols= teacher)
CD_S =  pd.read_csv("~/data/censo/2011/DADOS/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols= teacher)

#All teacher census data
CT = pd.concat([CD_CO, CD_NE, CD_N, CD_SE, CD_S])

In [42]:
ENEM = E.copy()
C_SCHOOL = CE.copy()
C_TEACHER = CT.copy()

In [43]:
#ordering
ENEM = ENEM[enem]

newNames = [
'CO_ANO',
'CO_ESCOLA',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'TP_ST_CONCLUSAO',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL'
]

#Rename
ENEM.columns = newNames

print('checking year',ENEM.CO_ANO.unique())
print('checking key',ENEM.CO_ESCOLA.dtypes)

checking year [2011]
checking key float64


In [44]:
C_SCHOOL = C_SCHOOL[school]

newNames = ['ANO_CENSO',
'CO_ESCOLA',
 'CO_MUNICIPIO',
 'CO_UF',
 'CO_DEPENDENCIA_ADM',
 'IN_AGUA_INEXISTENTE',
 'IN_ESGOTO_INEXISTENTE',
 'IN_ENERGIA_INEXISTENTE',
 'IN_LABORATORIO_INFORMATICA',
 'IN_LABORATORIO_CIENCIAS',
 'IN_SALA_ATENDIMENTO_ESPECIAL',
 'IN_BIBLIOTECA',
 'IN_SALA_LEITURA',
 'IN_BANHEIRO_FORA_PREDIO',
 'IN_BANHEIRO_PNE',
 'IN_DEPENDENCIAS_PNE',
 'QT_SALAS_UTILIZADAS',
 'QT_EQUIP_TV',
 'QT_EQUIP_DVD',
 'QT_EQUIP_COPIADORA',
 'QT_EQUIP_IMPRESSORA',
 'QT_COM_ADMINISTRATIVO',
 'QT_COMP_ALUNO',
 'IN_INTERNET',
 'IN_BANDA_LARGA',
 'QT_FUNCIONARIOS',
 'IN_ALIMENTACAO'
]

#Rename 
C_SCHOOL.columns =  newNames

print('Checking year of censo',C_SCHOOL.ANO_CENSO.unique())
C_SCHOOL.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_SCHOOL.CO_ESCOLA.dtypes)

Checking year of censo [2011]
checking key int64


In [45]:
C_TEACHER = C_TEACHER[teacher]

# New default column names for all Datasets
newNames = [
'CO_PROFESSOR',
'CO_ESCOLA',  
'IN_ESPECIALIZACAO',
'IN_MESTRADO',
'IN_DOUTORADO'
]
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
C_TEACHER.columns = newNames
C_TEACHER.drop_duplicates(subset=['CO_PROFESSOR', 'CO_ESCOLA'], inplace=True)
C_TEACHER.drop('CO_PROFESSOR', axis=1, inplace = True)

print('checking key',C_TEACHER.CO_ESCOLA.dtypes)

checking key int64


### Minimum Scope Definition

In [46]:
ENEM.dropna(inplace = True, axis=0, subset = ['CO_ESCOLA'])


In [47]:
#should care about the nulls into scope filter features?
print(ENEM['TP_ENSINO'].value_counts())
print(ENEM['TP_ENSINO'].isnull().sum())
#NO, IT'S LOWER

1.0    1338733
2.0     150626
4.0       9871
Name: TP_ENSINO, dtype: int64
1


In [48]:
print('drop out',((ENEM.TP_ENSINO!= 1)|(ENEM.TP_ENSINO!= 3)).sum(), 'who not attend regular and prof approachs schools')
ENEM = ENEM.loc[((ENEM.TP_ENSINO== 1)|(ENEM.TP_ENSINO== 3))]
ENEM.drop('TP_ENSINO', inplace = True, axis =1)

drop out 1499231 who not attend regular and prof approachs schools


In [49]:
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
for i in fields:
    print ('null:', i, ENEM[i].isnull().sum())
    print ('zero:', i, (ENEM[i]== 0).sum())
    
    #, "and", ENEM.I==0)

null: NU_NOTA_CN 175344
zero: NU_NOTA_CN 0
null: NU_NOTA_CH 175344
zero: NU_NOTA_CH 0
null: NU_NOTA_LC 195994
zero: NU_NOTA_LC 0
null: NU_NOTA_MT 195994
zero: NU_NOTA_MT 0
null: NU_NOTA_REDACAO 0
zero: NU_NOTA_REDACAO 248438


In [50]:
ENEM.dropna(inplace = True, axis=0, subset=fields)
ENEM = ENEM.loc[~(ENEM[fields] == 0).any(axis=1)]

In [51]:
#ONLY STUDENTS AT  17 -19 YEAR OLD. WE WANT TO GET ONLY REGULAR STUDENTS THAT ARE IN LAST YEAR OF SECONDARY SCHOOL. 
#THE MICRODADOS HAVE A FIELD TO CONTROL IT. BUT IT IS NOT CONFIDENCE,DUE A TURN OUT TEAACHERS THAT EXTEND ACADEMIC YEAR
print('null', ENEM.NU_IDADE.isnull().sum())
ENEM.NU_IDADE = ENEM.NU_IDADE.astype('int32')
#firts, fill NA with the mean of student school
ENEM.NU_IDADE = ENEM.groupby(ENEM['CO_ESCOLA'])['NU_IDADE'].apply(lambda x: x.fillna(round(x.mean())))
print('taking off',((ENEM.NU_IDADE <17)|(ENEM.NU_IDADE>19)).sum())
ENEM = ENEM.loc[(ENEM.NU_IDADE >16) &  (ENEM.NU_IDADE<20)]
ENEM.NU_IDADE.value_counts()

null 0
taking off 135154


17    559498
18    304513
19     87846
Name: NU_IDADE, dtype: int64

In [52]:
ENEM[fields].describe()

NU_NOTA_CN     NU_NOTA_CH     NU_NOTA_LC     NU_NOTA_MT  \
count  951857.000000  951857.000000  951857.000000  951857.000000   
mean      472.922855     480.129782     527.131950     531.099226   
std        85.790685      80.651046      72.939292     119.239856   
min       269.000000     252.900000     301.200000     321.600000   
25%       411.200000     422.900000     481.100000     432.500000   
50%       469.600000     481.000000     532.600000     520.800000   
75%       529.700000     535.700000     577.800000     615.300000   
max       867.200000     793.100000     795.500000     953.000000   

       NU_NOTA_REDACAO  
count    951857.000000  
mean        556.679180  
std         142.503534  
min          40.000000  
25%         460.000000  
50%         560.000000  
75%         640.000000  
max        1000.000000

### Transform features to default categories

In [53]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL']
ENEM[fields].isnull().sum()

TP_SEXO            0
TP_COR_RACA        0
EDU_PAI            0
EDU_MAE            0
QT_PESSOAS_CASA    0
RENDA_MENSAL       0
dtype: int64

In [54]:
for i in fields:
    print(i,ENEM[i].value_counts().index)

TP_SEXO Int64Index([1, 0], dtype='int64')
TP_COR_RACA Float64Index([1.0, 3.0, 2.0, 0.0, 4.0, 5.0], dtype='float64')
EDU_PAI Index(['B', 'E', 'C', 'G', 'I', 'D', 'F', 'H', 'A'], dtype='object')
EDU_MAE Index(['E', 'C', 'B', 'G', 'D', 'H', 'F', 'I', 'A'], dtype='object')
QT_PESSOAS_CASA Int64Index([4, 5, 3, 6, 2, 7, 8, 9, 1, 10, 11, 12, 20, 13, 14, 15, 16, 17, 18,
            19],
           dtype='int64')
RENDA_MENSAL Index(['D', 'C', 'B', 'E', 'F', 'G', 'H', 'J', 'I', 'A', 'K'], dtype='object')


In [55]:
#Order relation, is better for numeric categories.
categorical = ['EDU_PAI', 'EDU_MAE', 'RENDA_MENSAL']
for i in categorical:
    codes, uniques = pd.factorize(ENEM[i], sort=True)
    ENEM[i] = codes
    

In [56]:
ENEM['EDU_PAI_TEMP'] = ENEM['EDU_PAI']
ENEM['EDU_MAE_TEMP'] = ENEM['EDU_MAE']
ENEM['QT_PESSOAS_CASA_TEMP'] = ENEM['QT_PESSOAS_CASA']
ENEM['RENDA_MENSAL_TEMP'] = ENEM['RENDA_MENSAL']
ENEM['TP_SEXO_TEMP'] = ENEM['TP_SEXO']

In [57]:
# Paring with default categories

ENEM.loc[ENEM['EDU_PAI']==0, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==1, 'EDU_PAI_TEMP']=1
ENEM.loc[ENEM['EDU_PAI']==2, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==3, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==4, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==5, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==6, 'EDU_PAI_TEMP']=4
ENEM.loc[ENEM['EDU_PAI']==7, 'EDU_PAI_TEMP']=5
ENEM.loc[ENEM['EDU_PAI']==8, 'EDU_PAI_TEMP']=0


ENEM.loc[ENEM['EDU_MAE']==0, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==1, 'EDU_MAE_TEMP']=1
ENEM.loc[ENEM['EDU_MAE']==2, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==3, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==4, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==5, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==6, 'EDU_MAE_TEMP']=4
ENEM.loc[ENEM['EDU_MAE']==7, 'EDU_MAE_TEMP']=5
ENEM.loc[ENEM['EDU_MAE']==8, 'EDU_MAE_TEMP']=0


ENEM.loc[ENEM['QT_PESSOAS_CASA']==1, 'QT_PESSOAS_CASA_TEMP']=0
ENEM.loc[ENEM['QT_PESSOAS_CASA']==2, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==3, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==4, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==5, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==6, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==7, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']>7, 'QT_PESSOAS_CASA_TEMP']=3




ENEM.loc[ENEM['RENDA_MENSAL']==0, 'RENDA_MENSAL_TEMP']=0
ENEM.loc[ENEM['RENDA_MENSAL']==1, 'RENDA_MENSAL_TEMP']=1
ENEM.loc[ENEM['RENDA_MENSAL']==2, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==3, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==4, 'RENDA_MENSAL_TEMP']=3
ENEM.loc[ENEM['RENDA_MENSAL']==5, 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']==6, 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']>6, 'RENDA_MENSAL_TEMP']=5

ENEM.loc[ENEM['TP_SEXO']==1, 'TP_SEXO_TEMP']=0
ENEM.loc[ENEM['TP_SEXO']==0, 'TP_SEXO_TEMP']=1


In [58]:
#back to original columns
ENEM['EDU_PAI'] = ENEM['EDU_PAI_TEMP']
ENEM['EDU_MAE'] = ENEM['EDU_MAE_TEMP']
ENEM['QT_PESSOAS_CASA'] = ENEM['QT_PESSOAS_CASA_TEMP']
ENEM['RENDA_MENSAL'] = ENEM['RENDA_MENSAL_TEMP']
ENEM['TP_SEXO'] = ENEM['TP_SEXO_TEMP']



In [59]:
#drop temps

ENEM.drop(['EDU_PAI_TEMP', 'EDU_MAE_TEMP', 'QT_PESSOAS_CASA_TEMP','RENDA_MENSAL_TEMP', 'TP_SEXO_TEMP'], axis=1, inplace=True)

In [60]:
print('Total Students at ENEM:', ENEM.shape[0], 'of', ENEM.drop_duplicates('CO_ESCOLA').shape[0], 'schools')
print('Total Schools at Census:', C_SCHOOL.shape[0])

    
#######################################
print('########### Only about Schools at ENEM:')

schoolsEnem = ENEM['CO_ESCOLA'].dropna().unique()
schoolsCenso = C_SCHOOL['CO_ESCOLA'].dropna().unique()
schoolsTeacher = C_TEACHER['CO_ESCOLA'].dropna().unique()
schoolsEnem2 = ENEM['CO_ESCOLA'].dropna()

enem_censo = np.setdiff1d(schoolsEnem ,schoolsCenso)
print('Schools at ENEM out of School Census', sum(np.isin(schoolsEnem, schoolsCenso, invert=True)))

print('Schools at ENEM out of Teacher Census', sum(np.isin(schoolsEnem, schoolsTeacher, invert=True)))
print('Schools at ENEM and School Census out of Teacher Census', sum(np.isin(enem_censo, schoolsTeacher)))

print('Students Wasted*****', sum(np.isin(schoolsEnem2, schoolsCenso, invert=True)))


Total Students at ENEM: 951857 of 26840 schools
Total Schools at Census: 257880
########### Only about Schools at ENEM:
Schools at ENEM out of School Census 239
Schools at ENEM out of Teacher Census 601
Schools at ENEM and School Census out of Teacher Census 0
Students Wasted***** 2767


### New Features

ENEM

In [61]:
#Final Media
ENEM['NU_NOTA_GERAL'] = (ENEM.NU_NOTA_CN + ENEM.NU_NOTA_CH + ENEM.NU_NOTA_LC + ENEM.NU_NOTA_MT + ENEM.NU_NOTA_REDACAO)/5

SCHOOL CENSUS - Itś better include after merge all datasets years.

TEACHER CENSUS

In [62]:
C_TEACHER.isnull().sum()

CO_ESCOLA            0
IN_ESPECIALIZACAO    0
IN_MESTRADO          0
IN_DOUTORADO         0
dtype: int64

TEACHER CENSUS

In [63]:
#lets fill with lower level

C_TEACHER['IN_ESPECIALIZACAO'].fillna(0, inplace = True)
C_TEACHER['IN_MESTRADO'].fillna(0, inplace = True)
C_TEACHER['IN_DOUTORADO'].fillna(0, inplace = True)

In [64]:
C_TEACHER['TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 1

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_DOUTORADO ==1.0), 'TITULACAO'] = 3

#Rebuild level education information as indicator of unique level.
C_TEACHER['NU_GRADUACAO'] = 0
C_TEACHER['NU_ESPECIALIZACAO'] = 0
C_TEACHER['NU_MESTRADO'] = 0
C_TEACHER['NU_DOUTORADO'] = 0

C_TEACHER.loc[C_TEACHER.TITULACAO == 0, 'NU_GRADUACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 1, 'NU_ESPECIALIZACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 2, 'NU_MESTRADO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 3, 'NU_DOUTORADO']= 1


C_TEACHER['NU_GRADUACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_GRADUACAO'].transform(sum)
C_TEACHER['NU_ESPECIALIZACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_ESPECIALIZACAO'].transform(sum)
C_TEACHER['NU_MESTRADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_MESTRADO'].transform(sum)
C_TEACHER['NU_DOUTORADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_DOUTORADO'].transform(sum)



#Calculating weighted indice of teacher education
SUM= C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform(sum)
COUNT =  C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform('count')
C_TEACHER['TITULACAO'] = (SUM)/(COUNT*3)  

##arranjar isso dai



#transformin to school grain
C_TEACHER.drop_duplicates('CO_ESCOLA', inplace = True)

## STUDENT GRAIN

In [65]:
# teacher to Schoool Grain
print('C_SCHOOL',C_SCHOOL.shape)
print('C_TEACHER',C_TEACHER.shape)
CENSO19 = pd.merge(C_SCHOOL, C_TEACHER, on = 'CO_ESCOLA', how = 'inner')
print('C_SCHOOL + C_TEACHER',CENSO19.shape)

#Now, all to Student Grain
print('ENEM',ENEM.shape)
ENEM_CENSO_19 = pd.merge(ENEM, CENSO19, on = 'CO_ESCOLA', how = 'inner')
print('FINAL',ENEM_CENSO_19.shape)

ENEM_CENSO_19.to_csv('STUDENT.csv')

C_SCHOOL (257880, 26)
C_TEACHER (194932, 9)
C_SCHOOL + C_TEACHER (194932, 34)
ENEM (951857, 16)
FINAL (947191, 49)


## SCHOOL GRAIN

In [66]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL', 'NU_IDADE', 'NU_NOTA_CN',
             'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','NU_NOTA_GERAL' ]

for i in fields:   
    
            ENEM[i] = ENEM.groupby('CO_ESCOLA')[i].transform('mean')
            

In [67]:
# SCHOOL GRAIN AND SHOW THE NEW NOTA DISTRIBUTION
ENEM.drop_duplicates('CO_ESCOLA', inplace=True)
#print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

In [68]:
#recalculating target for new  Nota Geral distribution 
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

75.0 % lowers quartis


In [69]:
print('censo before',CENSO19.shape)
print('enem_school before', ENEM.shape)
CENSO_ENEM_19 = pd.merge(CENSO19, ENEM, on='CO_ESCOLA', how='inner')
print(CENSO_ENEM_19.shape)

CENSO_ENEM_19.to_csv('SCHOOL.csv', index=False)


censo before (194932, 34)
enem_school before (26840, 17)
(26239, 50)


In [70]:
CENSO_ENEM_19.head()

CO_ESCOLA  CO_MUNICIPIO  CO_UF  CO_DEPENDENCIA_ADM  IN_AGUA_INEXISTENTE  \
0   11000350       1100205     11                   4                  0.0   
1   11000384       1100205     11                   2                  0.0   
2   11017201       1100155     11                   2                  0.0   
3   11017384       1101559     11                   2                  0.0   
4   11017422       1100155     11                   2                  0.0   

   IN_ESGOTO_INEXISTENTE  IN_ENERGIA_INEXISTENTE  IN_LABORATORIO_INFORMATICA  \
0                    0.0                     0.0                           1   
1                    0.0                     0.0                           1   
2                    0.0                     0.0                           1   
3                    0.0                     0.0                           0   
4                    0.0                     0.0                           1   

   IN_LABORATORIO_CIENCIAS  IN_SALA_ATENDIMENTO_ESPECIAL  IN_BIBLIOTECA  \
0                        1                             0              1   
1                        0                             0              1   
2                        0                             1              1   
3                        1                             0              1   
4                        0                             0              0   

   IN_SALA_LEITURA  IN_BANHEIRO_FORA_PREDIO  IN_BANHEIRO_PNE  \
0                1                        1                1   
1                0                        0                1   
2                0                        0                0   
3                0                        0                0   
4                0                        0                0   

   IN_DEPENDENCIAS_PNE  QT_SALAS_UTILIZADAS  QT_EQUIP_TV  QT_EQUIP_DVD  \
0                    0                 14.0          1.0           1.0   
1                    0                 10.0          1.0           1.0   
2                    1                 10.0          1.0           1.0   
3                    0                  7.0          1.0           1.0   
4                    0                 10.0          1.0           1.0   

   QT_EQUIP_COPIADORA  QT_EQUIP_IMPRESSORA  QT_COM_ADMINISTRATIVO  \
0                 0.0                  1.0                   12.0   
1                 0.0                  1.0                    6.0   
2                 1.0                  1.0                   15.0   
3                 1.0                  1.0                    5.0   
4                 1.0                  1.0                    7.0   

   QT_COMP_ALUNO  IN_INTERNET  IN_BANDA_LARGA  QT_FUNCIONARIOS  \
0           25.0          1.0             1.0             67.0   
1           10.0          1.0             1.0             74.0   
2           10.0          1.0             0.0             93.0   
3           12.0          1.0             1.0             52.0   
4           10.0          1.0             1.0             33.0   

   IN_ALIMENTACAO  IN_ESPECIALIZACAO  IN_MESTRADO  IN_DOUTORADO  TITULACAO  \
0             0.0                  0            0             0   0.049020   
1             1.0                  0            0             0   0.137931   
2             1.0                  0            0             0   0.184211   
3             1.0                  1            0             0   0.274510   
4             1.0                  1            0             0   0.187500   

   NU_GRADUACAO  NU_ESPECIALIZACAO  NU_MESTRADO  NU_DOUTORADO  CO_ANO  \
0            29                  5            0             0    2011   
1            19                  8            2             0    2011   
2            17                 21            0             0    2011   
3             3                 14            0             0    2011   
4             7                  9            0             0    2011   

    NU_IDADE   TP_SEXO  TP_COR_RACA  TP_ST_CONCLUSAO

In [71]:
CENSO_ENEM_19.isnull().sum()


CO_ESCOLA                          0
CO_MUNICIPIO                       0
CO_UF                              0
CO_DEPENDENCIA_ADM                 0
IN_AGUA_INEXISTENTE                0
IN_ESGOTO_INEXISTENTE              0
IN_ENERGIA_INEXISTENTE             0
IN_LABORATORIO_INFORMATICA         0
IN_LABORATORIO_CIENCIAS            0
IN_SALA_ATENDIMENTO_ESPECIAL       0
IN_BIBLIOTECA                      0
IN_SALA_LEITURA                    0
IN_BANHEIRO_FORA_PREDIO            0
IN_BANHEIRO_PNE                    0
IN_DEPENDENCIAS_PNE                0
QT_SALAS_UTILIZADAS                0
QT_EQUIP_TV                        0
QT_EQUIP_DVD                       0
QT_EQUIP_COPIADORA                 0
QT_EQUIP_IMPRESSORA                0
QT_COM_ADMINISTRATIVO            521
QT_COMP_ALUNO                   2058
IN_INTERNET                      327
IN_BANDA_LARGA                  1373
QT_FUNCIONARIOS                    0
IN_ALIMENTACAO                     0
IN_ESPECIALIZACAO                  0
I